In [65]:
import pandas as pd
import datetime

# from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from sklearn.metrics import log_loss
import xgboost
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)

OSError: dlopen(/Users/yuhua/anaconda2/lib/python2.7/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/gcc/lib/gcc/7/libgomp.1.dylib
  Referenced from: /Users/yuhua/anaconda2/lib/python2.7/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [2]:
import sklearn
sklearn.__version__

'0.19.0'

In [3]:
train_df = pd.read_table('/Users/yuhua/先验CTR模型线上观察/Alimama比赛/round1_ijcai_18_train_20180301.txt',sep=' ')


In [4]:
train_df['item_id'].value_counts()

7571023501622243456    3001
2986722878397696833    2836
5410526681843914464    2578
2079092401639430105    2515
2709839111509701702    2448
5716046650492250518    2267
5467735841796001072    2258
2723541945574109807    2021
3444647220397515150    1937
4817583173619024436    1919
4162406225225963765    1780
7937690818610877888    1683
8970449145125394567    1631
534920639711707822     1608
751443051994527892     1596
671302156973073367     1591
5634978682268415449    1571
5586531724608162781    1528
2349124492995958413    1403
2530039332833674681    1397
509936336103770855     1396
5088920748697829728    1377
4885486757349832183    1368
3401173841796395052    1339
4429962171977692955    1321
7697581527441910922    1260
8906948455106358440    1249
8961114074287787809    1245
6335890202072992021    1219
7085665113142227362    1197
8777881594532200391    1174
365202798058356244     1171
3369991836196154735    1166
5687838372958632135    1162
7531396996295949220    1161
7028360717147795655 

In [5]:
test_df = pd.read_table('/Users/yuhua/先验CTR模型线上观察/Alimama比赛/round1_ijcai_18_test_a_20180301.txt',sep=' ')

In [6]:
train_df.columns


Index([u'instance_id', u'item_id', u'item_category_list',
       u'item_property_list', u'item_brand_id', u'item_city_id',
       u'item_price_level', u'item_sales_level', u'item_collected_level',
       u'item_pv_level', u'user_id', u'user_gender_id', u'user_age_level',
       u'user_occupation_id', u'user_star_level', u'context_id',
       u'context_timestamp', u'context_page_id', u'predict_category_property',
       u'shop_id', u'shop_review_num_level', u'shop_review_positive_rate',
       u'shop_star_level', u'shop_score_service', u'shop_score_delivery',
       u'shop_score_description', u'is_trade'],
      dtype='object')

In [7]:
train_df_ready = train_df[['item_id', 
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_gender_id',
       'user_age_level', 'user_occupation_id','context_timestamp', 'user_star_level', 'context_page_id', 
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade']]

In [8]:
train_df_ready.count()

item_id                      478138
item_brand_id                478138
item_city_id                 478138
item_price_level             478138
item_sales_level             478138
item_collected_level         478138
item_pv_level                478138
user_gender_id               478138
user_age_level               478138
user_occupation_id           478138
context_timestamp            478138
user_star_level              478138
context_page_id              478138
shop_id                      478138
shop_review_num_level        478138
shop_review_positive_rate    478138
shop_star_level              478138
shop_score_service           478138
shop_score_delivery          478138
shop_score_description       478138
is_trade                     478138
dtype: int64

In [62]:
def train_and_test_xgboost(train_df, test_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    xgb = xgboost.XGBClassifier(nthread=20)
    xgb.fit(X_train, y_train)
    
    return log_loss(y_test,xgb.predict_proba(X_test))

def train_and_test_lr(train_df, test_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    
    return log_loss(y_test,lr.predict_proba(X_test))

def train_and_test_randomforest(train_df, test_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
#     rf = RandomForestClassifier(n_estimators=32, max_depth=40, min_samples_split=100, min_samples_leaf=10,  criterion='entropy',
#                         max_features=8, verbose = 1,  bootstrap=False, n_jobs=10)
#     RandomForestClassifier(criterion='entropy',n_estimators=100,n_jobs=15)

    rf = RandomForestClassifier(n_estimators=32, max_depth=20, min_samples_split=100, min_samples_leaf=10,  criterion='entropy', max_features=8, verbose = 1,  bootstrap=False)
    rf.fit(X_train, y_train)
    
    return log_loss(y_test,rf.predict_proba(X_test))

def train_and_test_gbdt(train_df, test_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    gbdt = GradientBoostingClassifier()
    gbdt.fit(X_train, y_train)
    
    return log_loss(y_test,gbdt.predict_proba(X_test))

def train_and_test_gbdt(train_df, test_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    gbdt = GradientBoostingClassifier()
    gbdt.fit(X_train, y_train)
    
    return log_loss(y_test,gbdt.predict_proba(X_test))

In [63]:
%run BaseFrame.py

tat = BaseFrame(train_df_ready, 0.95,'context_timestamp', True)
print "xgboost=",tat.kflod_validation_seq(10,train_and_test_xgboost)

tat = BaseFrame(train_df_ready, 0.95,'context_timestamp', True)
print "lr=",tat.kflod_validation_seq(10,train_and_test_lr)

tat = BaseFrame(train_df_ready, 0.95,'context_timestamp', True)
print "randomforest=",tat.kflod_validation_seq(10,train_and_test_randomforest)

tat = BaseFrame(train_df_ready, 0.95,'context_timestamp', True)
print "gbdt=",tat.kflod_validation_seq(10,train_and_test_gbdt)

xgboost= 0.0925939014178
lr= 0.103369666454
randomforest=

[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    2.9s finished
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    3.4s finished
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    2.8s finished
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    1.7s finished
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    2.8s finished
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    3.1s finished
[Parallel(n_jobs=1)]

 0.107551474962
gbdt= 0.0974024865334
